# Tool Calling with Azure OpenAI: gpt-4o-mini

In [1]:
from openai import AzureOpenAI

In [2]:
client = AzureOpenAI()

In [3]:
import requests,json
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output

In [4]:
get_current_weather("mumbai")

{'City Name': 'mumbai',
 'weather': 'broken clouds',
 'temperature': 299.7,
 'unit': 'kelvin'}

### Tool Integration with LLM
- metadata
    - name of tool/function
    - description of the tool/function
    - arguments details - arguments and its description for the tool/function

In [11]:
tools = [{
    "type":"function",
    "function":{
        "name":"get_current_weather",
        "description":"this function can be used to get the current weather information for any given city.",
        "parameters":{
            "type":"object",
            "properties":{"city":{"type":"string","description":"name of city e.g. mumbai, new york"}},
        },
        "required":['city'],
    }}
]

In [18]:
tool_map={"get_current_weather":get_current_weather}

def generate_response(prompt):

    messages = [{"role":"system","content":"you are a helpful assistant"},
                {"role":"user",'content':prompt}]
    
    first_response = client.chat.completions.create(model='gpt4o',
                                                    messages=messages,
                                                    temperature=0.2,
                                                    tools=tools,tool_choice='auto')
    if first_response.choices[0].message.tool_calls:
        # do something
        print("LLM decided to trigger tool call",first_response.choices[0].message.tool_calls)
        tool_calls = first_response.choices[0].message.tool_calls
        messages.append(first_response.choices[0].message)
        for tool in tool_calls:
            tool_name=tool.function.name
            tool_args = json.loads(tool.function.arguments)
            fun_to_ex = tool_map[tool_name]
            tool_output = fun_to_ex(**tool_args)
            messages.append({"tool_call_id":tool.id,"role":"tool",
                            "name":tool_name,"content":json.dumps(tool_output)})
        second_response = client.chat.completions.create(model='gpt4o',messages=messages,
                                                         temperature=0.1)
        return second_response.choices[0].message.content

    else:
        return first_response.choices[0].message.content

In [19]:
generate_response("What is Artificial Intelligence")

'Artificial Intelligence (AI) refers to the simulation of human intelligence processes by computer systems. These processes include learning (the acquisition of information and rules for using it), reasoning (using rules to reach approximate or definite conclusions), and self-correction. AI can be categorized into two main types:\n\n1. **Narrow AI (Weak AI)**: This type of AI is designed and trained for a specific task. Examples include virtual assistants like Siri and Alexa, recommendation systems, and image recognition software.\n\n2. **General AI (Strong AI)**: This type of AI would have the ability to understand, learn, and apply intelligence to solve any problem, much like a human. General AI remains largely theoretical and has not yet been achieved.\n\nAI technologies encompass various subfields, including:\n\n- **Machine Learning**: A subset of AI that enables systems to learn from data and improve their performance over time without being explicitly programmed.\n- **Natural Lan

In [20]:
generate_response("what is the current weather in delhi today?")

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_eU5zGoXvBHoPvPFHrenNC35b', function=Function(arguments='{"city":"delhi"}', name='get_current_weather'), type='function')]


'The current weather in Delhi today is clear sky with a temperature of approximately 305.25 K (which is about 32.1°C).'

In [11]:
result = generate_response("what is the current weather in delhi and bangalore today?")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_iIFuDh3UFd4nrUCST1aPX0MK', function=Function(arguments='{"city": "delhi"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_iDQanrbjvLa2UTqnSt9AInLP', function=Function(arguments='{"city": "bangalore"}', name='get_current_weather'), type='function')]
As of today, the weather is as follows:

- **Delhi**: Clear sky with a temperature of approximately 305.11 K (about 32°C).
- **Bangalore**: Broken clouds with a temperature of approximately 299.15 K (about 26°C). 

If you need more specific details or forecasts, feel free to ask!


In [12]:
result = generate_response("what is the current weather today?")
print(result)

Could you please specify the city for which you would like to know the current weather?


In [13]:
!python -m pip install wikipedia --quiet

In [14]:
import wikipedia

def get_wikipedia_summary(query:str):
    response = wikipedia.summary(query)
    return response

In [15]:
tools = [{
    "type":"function",
    "function":{
        "name":"get_current_weather",
        "description":"this function can be used to get the current weather information for any given city.",
        "parameters":{
            "type":"object",
            "properties":{"city":{"type":"string","description":"name of city e.g. mumbai, new york"}},
        },
        "required":['city'],
    }},
    {
    "type":"function",
    "function":{
        "name":"get_wikipedia_summary",
        "description":"this function can be used to get information about any place, location, person, historical information.",
        "parameters":{
            "type":"object",
            "properties":{"query":{"type":"string","description":"name of any person, event, location, place"}},
        },
        "required":['query'],
    }}
]

In [18]:
tool_map={"get_current_weather":get_current_weather,
          "get_wikipedia_summary":get_wikipedia_summary}

def generate_response(prompt):

    messages = [{"role":"system","content":"you are a helpful assistant"},
                {"role":"user",'content':prompt}]
    
    while True:
        response = client.chat.completions.create(model='gpt4o',
                                                        messages=messages,
                                                        temperature=0.2,
                                                        tools=tools,tool_choice='auto')
        if response.choices[0].message.tool_calls:
            # do something
            print("LLM decided to trigger tool call",response.choices[0].message.tool_calls)
            tool_calls = response.choices[0].message.tool_calls
            messages.append(response.choices[0].message)
            for tool in tool_calls:
                tool_name=tool.function.name
                tool_args = json.loads(tool.function.arguments)
                fun_to_ex = tool_map[tool_name]
                tool_output = fun_to_ex(**tool_args)
                messages.append({"tool_call_id":tool.id,"role":"tool",
                                "name":tool_name,"content":json.dumps(tool_output)})
        else:
            break
    return response.choices[0].message.content

In [19]:
result = generate_response("what is the current weather in mumbai?")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_CbvZxVwHyMk2j0aveNQp6oj9', function=Function(arguments='{"city":"mumbai"}', name='get_current_weather'), type='function')]
The current weather in Mumbai is characterized by broken clouds, with a temperature of approximately 299.19 K (which is about 26.04 °C or 78.87 °F).


In [20]:
result = generate_response("what is the current weather in mumbai and delhi?")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_NvIQ6Ps0cRq4Obpodgkd5qIG', function=Function(arguments='{"city": "mumbai"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_LRJFqiOiwkrmSjVY6Xl5Nicr', function=Function(arguments='{"city": "delhi"}', name='get_current_weather'), type='function')]
The current weather in the two cities is as follows:

- **Mumbai**: 
  - Weather: Broken clouds
  - Temperature: 299.19 K (approximately 26.04 °C)

- **Delhi**: 
  - Weather: Clear sky
  - Temperature: 305.11 K (approximately 32.96 °C)


In [21]:
result = generate_response("tell me something about Mumbai and its current weather")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_wKVR3mBKkwUZwfOuNAWb8Wt1', function=Function(arguments='{"query": "Mumbai"}', name='get_wikipedia_summary'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_6yHBoCnnP9Qfpc14gkaDC9Ic', function=Function(arguments='{"city": "Mumbai"}', name='get_current_weather'), type='function')]
### About Mumbai
Mumbai, formerly known as Bombay, is the capital city of the Indian state of Maharashtra. It is the financial capital and the most populous city in India, with an estimated population of 12.5 million. The city is part of the Mumbai Metropolitan Region, which has a population exceeding 23 million, making it one of the most populous metropolitan areas in the world.

Mumbai is located on the Konkan coast and has a deep natural harbor. Historically, the seven islands that make up Mumbai were inhabited by the Koli people and were under various rulers before being ceded to the Portuguese and later to th

In [25]:
result = generate_response("which is the larget city in United Arab Emirates and then tell me what is the weather there?")
print(result)

LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_MQm3Y1Qs6cvnXlPLVCsKuUdy', function=Function(arguments='{"query":"largest city in United Arab Emirates"}', name='get_wikipedia_summary'), type='function')]
LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_nFJEJ2DplJwaHgs69dhicDI9', function=Function(arguments='{"query":"Dubai"}', name='get_wikipedia_summary'), type='function')]
LLM decided to trigger tool call [ChatCompletionMessageFunctionToolCall(id='call_oAOsFqKb19ocyhb6FK9aOXFG', function=Function(arguments='{"city":"Dubai"}', name='get_current_weather'), type='function')]
The largest city in the United Arab Emirates is **Dubai**. 

Currently, the weather in Dubai is characterized by **few clouds**, with a temperature of approximately **306.58 K** (which is about **33.4°C**).
